In [91]:
import numpy as np # giving an alas

from model import calculate_fitness # to check fitness inside reverse learning

In [92]:
# these 2 varibles givin in the document already
a_chaos = 0.5
b_chaos = 2.2

In [93]:
# these the boundrys given
LOWER_BOUND = np.array([113.8, 22.45]) 
UPPER_BOUND = np.array([114.4, 22.85])

In [94]:
def circle_chaotic_map(dim, n_pop):
	
	z = np.random.rand(n_pop, dim)

	term1 = b_chaos / (2 * np.pi)
	term2 = np.sin(2 * np.pi * z) # using eq11

	new_z = z + a_chaos - (term1 * term2) # than assamble in 1 varible 
	chaotic_pop = new_z % 1 # to kkep it between 0 and 1
    
	# one single whale in our code represents 14 different stations all at once it makes an array of 28, but we have 2, x and y cordinates, we will stretch it 14 times to reach same width as whale array 	 
	lb_tiled = np.tile(LOWER_BOUND, int(dim/2))
	ub_tiled = np.tile(UPPER_BOUND, int(dim/2))
    
	scaled_pop = lb_tiled + chaotic_pop * (ub_tiled - lb_tiled) # scaling part 
    
	return scaled_pop

In [95]:
def apply_reverse_learning(population, fitness_scores, demand_points):

	n_pop = population.shape[0]
	dim = population.shape[1]

	# one single whale in our code represents 14 different stations all at once it makes an array of 28, but we have 2, x and y cordinates, we will stretch it 14 times to reach same width as whale array
	lb_tiled = np.tile(LOWER_BOUND, int(dim/2))
	ub_tiled = np.tile(UPPER_BOUND, int(dim/2))

	# paper say we have to sort the population acoording to the fitness value from best lower cost to worst higher cost
	sorted_indices = np.argsort(fitness_scores)
	sorted_pop = population[sorted_indices]
	sorted_fitness = fitness_scores[sorted_indices]

	# ratio is 1 3 6
	limit_layer1 = int(n_pop * 0.1) # cut top 10
	limit_layer2 = int(n_pop * 0.4) # cut top 40 (30 + 10) 10 coming from layer 1 
    
	# giving them to new varibles
	new_pop = sorted_pop.copy()
	new_fitness = sorted_fitness.copy()

	for i in range(n_pop): # this is the top part we do nothing to them they are ok
		if i < limit_layer1:
			continue

		original_whale = sorted_pop[i]
		reverse_whale = (lb_tiled + ub_tiled) - original_whale # we are calculating opposite side of the map by eq13
        
		reverse_whale = np.clip(reverse_whale, lb_tiled, ub_tiled) # this will ensure we are not outside of the map
        
		stations_reshaped = reverse_whale.reshape(14, 2) # ok we got new position reshaped
		rev_fit = calculate_fitness(stations_reshaped, demand_points) # but is it any better
        
		if limit_layer1 <= i < limit_layer2: # for middle layer we compare it old fitness with new calculated one, if new one better we replace it, eq14 says so
			if rev_fit < sorted_fitness[i]:
				new_pop[i] = reverse_whale
				new_fitness[i] = rev_fit
                
		else: # aha bottom layer, this time we will caculate opposite but do not check if its better because its already bad 
			new_pop[i] = reverse_whale
			new_fitness[i] = rev_fit
            
		return new_pop, new_fitness

In [ ]:
# to check if its working 
if __name__ == "__main__":
    # Test Data
    n_pop = 30
    dim = 28 # 14 stations * 2
    
    # generate mock demand points for testing
    mock_demand = np.random.uniform(113.8, 114.4, (130, 2)) # X range

    print("Generating Population...")
    pop = circle_chaotic_map(dim, n_pop)
    print(f"Population generated. Shape: {pop.shape}")
    print(f"Sample Station (Lon, Lat): {pop[0,0:2]}") # Should be ~114, ~22
    
    # Calculate initial fitness
    fits = []
    for whale in pop:
        fits.append(calculate_fitness(whale.reshape(14,2), mock_demand))
    fits = np.array(fits)
    
    print("\nApplying Reverse Learning...")
    new_pop, new_fits = apply_reverse_learning(pop, fits, mock_demand)
    
    print(f"Best Fitness Before: {np.min(fits):.2f}")
    print(f"Best Fitness After:  {np.min(new_fits):.2f}") # after one must be lower or same 

Generating Population...
Population generated. Shape: (30, 28)
Sample Station (Lon, Lat): [114.1599548   22.81984209]

Applying Reverse Learning...
Best Fitness Before: 783377022.88
Best Fitness After:  783377022.88
